# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Отсутствуют значения только в столбцах 'days_employed' и 'total_income'.
Столбцам 'days_employed' и 'total_income' стоит присвоить целочисленные значения (текущая точность нам не нужна)
Получим общую информацию:

Проверим, это одни и те же строки, где отсутствуют данные по обоим столбцам ('days_employed' и 'total_income') или нет.

In [3]:
data[(data['total_income'].isnull() == True) & (data['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Предположение подтвердилось - в строках, где отсутствуют данные в столбце 'days_employed', отсутствуют данные и по 'total_income'.
Проверим, из разных ли это профессий люди (если из одной, то можно будет заполнить NaN средним по этой группе).

In [4]:
data[(data['total_income'].isnull() == True) & (data['days_employed'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Из разных. Значит, взять среднее по одной группе профессий ('income_type') уже нельзя, будем заполнять NaN по медиане из каждой группы.

In [5]:
#Найдем долю пропущенных значений в 'days_employed'
total_lines_days_employed = len(data['days_employed'].axes[0])
total_values_days_employed = data['days_employed'].count()
conversion_days_employed = (total_lines_days_employed - total_values_days_employed) / total_lines_days_employed
print(conversion_days_employed)

0.10099883855981417


In [6]:
#Найдем долю пропущенных значений в 'total_income'
total_lines_total_income = len(data['total_income'].axes[0])
total_values_total_income = data['total_income'].count()
conversion_total_income = (total_lines_total_income - total_values_total_income)/total_lines_total_income
print(conversion_total_income)

0.10099883855981417


In [7]:
#Заполним пропуски 'total_income'
median_for_total_income = data.pivot_table(index='income_type',  values = 'total_income',  aggfunc='median')
data['median_total_income'] = data.apply(lambda row: median_for_total_income['total_income'][row['income_type']], axis=1)
data['total_income'] =data['total_income'].fillna(data['median_total_income'])



In [8]:
#Заполним пропуски 'days_employed'
median_for_days_employed = data.pivot_table(index='income_type',  values = 'days_employed',  aggfunc='median')
data['median_days_employed'] = data.apply(lambda row: median_for_days_employed['days_employed'][row['income_type']], axis=1)
data['days_employed'] =data['days_employed'].fillna(data['median_days_employed'])

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21525 non-null  int64  
 1   days_employed         21525 non-null  float64
 2   dob_years             21525 non-null  int64  
 3   education             21525 non-null  object 
 4   education_id          21525 non-null  int64  
 5   family_status         21525 non-null  object 
 6   family_status_id      21525 non-null  int64  
 7   gender                21525 non-null  object 
 8   income_type           21525 non-null  object 
 9   debt                  21525 non-null  int64  
 10  total_income          21525 non-null  float64
 11  purpose               21525 non-null  object 
 12  median_total_income   21525 non-null  float64
 13  median_days_employed  21525 non-null  float64
dtypes: float64(4), int64(5), object(5)
memory usage: 2.3+ MB


данные могут отсутствовать так как не все пишут свои доходы, хотя это кредит. а данные отсутствия трудового стажа, тк может быть новая трудовая книжка.


Трудовой стаж и ежемесячный доход это количественные переменные.Пропуски в таких переменных заполняют характерными значениями.
чтобы примерно оценить типичные значения выборки годится медиана или средне арифметическое. Нам больше подходит медиана, так как 
некорректно считать по среднеарифметическому средний стаж и доход.

### Шаг 2.2. Проверка данных на аномалии и исправления. 
 

 Мы видим что при первом выводе значения в столбце 'days_employed' отрицательные, исправим это


In [10]:
#Берем значения по 'days_employed'по модулю.
data['days_employed'] = data['days_employed'].abs()
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_total_income,median_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847,-1574.202821
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847,-1574.202821
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847,-1574.202821
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847,-1574.202821
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412,365213.306266


In [11]:
#Выведем уникальные значения по столбцу 'children'
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


Мы видим странные значения -1 ребенок и 20 детей, которые явно появились по какой-то ошибке. Исправим это.

In [12]:
#Посмотрим какое количество этих значений
print("Количество строк с 'children' -1 -", data[data['children'] == -1].count()[0])
print("Количество строк с 'children' 20 -", data[data['children'] == 20].count()[0])

Количество строк с 'children' -1 - 47
Количество строк с 'children' 20 - 76


Всего 47 строк со значением по количеству детей равному "-1", изменим на "0".
Те, у кого 20 детей, впоследствии отнесен к многодетным (>2 детей).

In [13]:
#Заменим данные с -1 на 0
data['children'] = data['children'].replace(-1, 0)

In [14]:
print(data['children'].unique())

[ 1  0  3  2  4 20  5]


In [15]:
#Выведем уникальные значения по столбцу 'dob_years'
print(data['dob_years'].unique())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


Мы видим странные значения: нулевой возраст по некоторым строкам 'dob_years'.
Обработаем эти значения

In [16]:
## смотрим количество нулей по возрасту (столбец 'dob_years')
print("Количество строк с 'dob_years' 0 -", data[data['dob_years'] == 0].count()[0])

Количество строк с 'dob_years' 0 - 101


In [17]:
#Заполним значения медианой по группе
age = data['dob_years'].median()
data=data.replace({'dob_years': {0: age}})

In [18]:
#Убедимся что все заемщики совершеннолетнии, выведем минимальный возраст в группе
print(data['dob_years'].min())

19.0


### Шаг 2.3. Изменение типов данных.

In [19]:
data['days_employed']=data['days_employed'].astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_total_income,median_days_employed
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847,-1574.202821
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847,-1574.202821
2,0,5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847,-1574.202821
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847,-1574.202821
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412,365213.306266


### Шаг 2.4. Удаление дубликатов.

In [20]:
data.duplicated().sum()

55

In [21]:
#Отсортируем дубликаты
data[data.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_total_income,median_days_employed
1005,0,365213,62.0,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,ремонт жилью,118514.486412,365213.306266
1191,0,365213,61.0,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,операции с недвижимостью,118514.486412,365213.306266
1511,0,365213,58.0,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514.486412,дополнительное образование,118514.486412,365213.306266
1681,0,365213,57.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,на проведение свадьбы,118514.486412,365213.306266
2052,0,365213,58.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу,118514.486412,365213.306266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10697,0,1547,40.0,среднее,1,гражданский брак,1,F,компаньон,0,172357.950966,сыграть свадьбу,172357.950966,-1547.382223
13878,1,1547,31.0,среднее,1,женат / замужем,0,F,компаньон,0,172357.950966,покупка жилья,172357.950966,-1547.382223
17379,0,1547,54.0,высшее,0,женат / замужем,0,M,компаньон,0,172357.950966,операции с коммерческой недвижимостью,172357.950966,-1547.382223
17774,1,1547,40.0,среднее,1,гражданский брак,1,F,компаньон,0,172357.950966,строительство жилой недвижимости,172357.950966,-1547.382223


In [22]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

-столбец 'education' имеет неоднотипные значения, необходимо привести к общему виду;

In [23]:
# 'education' в low-индекс
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [24]:
# дубликаты. Причина? Предполагаю, что задвоение данных/ошибочное создание дублей.
#Удаляем их.
data = data.drop_duplicates()

In [25]:
data.duplicated().sum()

0

In [26]:
data.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [27]:
#словарь для education
education_log = data[['education_id', 'education']]
# удалим дубликаты из словаря
education_log = education_log.drop_duplicates().reset_index(drop=True)
education_log.head(15)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [28]:
#словарь для family_status
family_status_log = data[['family_status_id', 'family_status']]
# удалим дубликаты из словаря
family_status_log = family_status_log.drop_duplicates().reset_index(drop=True)
family_status_log.head(15)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [29]:
data = data.drop(columns = ['education', 'family_status'], axis = 1)
data.head(5) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,median_total_income,median_days_employed
0,1,8437,42.0,0,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847,-1574.202821
1,1,4024,36.0,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847,-1574.202821
2,0,5623,33.0,1,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847,-1574.202821
3,3,4124,32.0,1,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847,-1574.202821
4,0,340266,53.0,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412,365213.306266


### Шаг 2.6. Категоризация дохода.

In [30]:
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income <=50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <=1000000:
        return 'B'
    if total_income >=1000001:
        return 'A'    
data['total_income_category'] = data['total_income'].apply(total_income_category)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,median_total_income,median_days_employed,total_income_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847,-1574.202821,B
1,1,4024,36.0,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847,-1574.202821,C
2,0,5623,33.0,1,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847,-1574.202821,C
3,3,4124,32.0,1,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847,-1574.202821,B
4,0,340266,53.0,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412,365213.306266,C


### Шаг 2.7. Категоризация целей кредита.

In [31]:

def purpose_category(purpose):
    if 'образов' in purpose:
        return 'получение образования'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'недвижимост' or 'жиль' in purpose:
        return 'операции с недвижимостью'    
data['purpose_category'] = data['purpose'].apply(purpose_category)
print(data['purpose'].unique())
print(data['purpose_category'].unique())
data.head(5)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']
['операции с недвижимостью' '

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,median_total_income,median_days_employed,total_income_category,purpose_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847,-1574.202821,B,операции с недвижимостью
1,1,4024,36.0,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847,-1574.202821,C,операции с автомобилем
2,0,5623,33.0,1,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847,-1574.202821,C,операции с недвижимостью
3,3,4124,32.0,1,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847,-1574.202821,B,получение образования
4,0,340266,53.0,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412,365213.306266,C,проведение свадьбы


### Ответы на вопросы.

##### Вопрос 1:Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [32]:
def children_cat(data):
    if data['children'] == 0:
        return 'нет детей'
    elif 1 <= data['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'
data['children_cat'] = data.apply(children_cat, axis=1)    
def relation(category):
    return data.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')    
relation('children_cat')


,debt
children_cat,
нет детей,0.075263
многодетные,0.085526
1-2 ребенка,0.093003


Заемщики, не имеющие детей, меньше просрачивают по выплатам кредита.

Вопрос 2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
data.groupby('family_status_id')['debt'].mean().to_frame().sort_values(by='debt')

,debt
family_status_id,
2,0.065693
3,0.071130
0,0.075452
1,0.093494
4,0.097509


Граждане, которые не имеют узаконенных отношений, более склонны к просрочке по выплатам кредита

 Вопрос 3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
def total_income_cat(data):
    if data['total_income'] <= 32000:
        return 'низкий доход'
    if 32000 < data['total_income'] <= 120000:
        return 'средний доход'
    if 120000 < data['total_income'] < 1000000:
        return 'высокий доход'
    return 'миллионер'
data['total_income_cat'] = data.apply(total_income_cat, axis=1)
def relation(category):
    return data.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')    
relation('total_income_cat')

,debt
total_income_cat,
низкий доход,0.058824
миллионер,0.080000
средний доход,0.081145
высокий доход,0.081214


Заемщики с уровнем дохода < 32000р. менее склонны нарушать обязательства по выплатам кредита в срок.

Вопрос №4 Как разные цели кредита влияют на его возврат в срок

In [35]:
relation('purpose_category')

,debt
purpose_category,
операции с недвижимостью,0.072334
проведение свадьбы,0.080069
получение образования,0.092200
операции с автомобилем,0.093590


In [36]:
pd.pivot_table(data,index=["purpose_category"],values=["debt"])

,debt
purpose_category,
операции с автомобилем,0.093590
операции с недвижимостью,0.072334
получение образования,0.092200
проведение свадьбы,0.080069


Заемщики с целью покупки недвижимости более склонны возвращать кредита в срок.

Семеное положение и количество детей влияет на факт погашения кредита в строк:
-заемщики с официально оформленными отношениями (или которые в прошлом были в официальном в браке) и не имеющие детей - самые ответственные заемщики.
-заемщики, состоящие в неофициальном браке или находящиеся без отношений, при этом имеющие 1 или 2 детей - самые менее ответственные заемщики.


## Общий вывод: 
* В ходе работы изучили общую информацию о данных банка.
Проделали предобработку данных. В ходе которых можно сделать такой вывод.
Действительно, семейное положение и количество детей клиента влияют на факт погашения кредита. Но еще стоит отметить, что большенство кредитов выплачиваются в срок с целью приобретения недвижимости и автокредита, менее всего на свадьбу. так же люди имеющие низкий уровень дохода более ответственно подходят к погашению кредита в срок. Из всего этого следует, что банку менее рисковано выдавать кредит людям с целью приобретения жилья.